In [1]:
from models_v4 import desperate_transformer
from dataset import ImageCaptionDataset
import pickle
from torch.utils.data import DataLoader
import wandb
import yaml
from typing import Dict
import gensim.downloader as api
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn

from tqdm import tqdm
from icecream import ic

import numpy as np

import os


###SCRIPT CONFIG!####
device = "mps" #r u running cuda my boy? or mps? :D
num_epochs = 200
batch_size = 50
print("Have you runned wandb login?? OK. go aheadd...")
#####################

#convert to dictionary a yaml 
def nested_dict(original_dict):
    nested_dict = {}
    for key, value in original_dict.items():
        parts = key.split(".")
        d = nested_dict
        for part in parts[:-1]:
            if part not in d:
                d[part] = {}
            d = d[part]
        d[parts[-1]] = value
    return nested_dict

#load datasets
with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/train_dataset.pkl', 'rb') as inp:
    train_dataset = pickle.load(inp)
with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/val_dataset.pkl', 'rb') as inp:
    val_dataset = pickle.load(inp)
#with open('/Users/josepsmachine/Documents/UNI/DL/dlnn-project_ia-group_10/dataset/debug_dataset.pkl', 'rb') as inp:
#    debug_dataset = pickle.load(inp)

#create dataloaders

#debug_dataloader = DataLoader(debug_dataset, batch_size=1, shuffle=False)


#load word2vec pretrained embedding layer
word2vec_emb = api.load('word2vec-google-news-300')
word2vec_emb = torch.FloatTensor(word2vec_emb.vectors)

#import vocabulary to word2vec indexes that we know
with open('vocabidx2word2vecidx.pkl', 'rb') as inp:
    vocabidx2word2vecidx = pickle.load(inp)

with open('vocabulary.pkl', 'rb') as inp:
    vocabulary = pickle.load(inp)

word2vec_emb = word2vec_emb[vocabidx2word2vecidx]

word2vec_emb = nn.Embedding.from_pretrained(word2vec_emb)
word2vec_emb.requires_grad_ = False #freeze word2vec embeddding layer

#load universal sentence encoder to define our own loss function
sntc_enc = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#define cross entropy loss function
cross_entrop = nn.CrossEntropyLoss(ignore_index=2074)

/Users/josepsmachine/miniforge3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Have you runned wandb login?? OK. go aheadd...


In [2]:
len(val_dataset)

8090

In [3]:
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [4]:
val_dataset.__getitem__(0)[0].shape

torch.Size([2048])

In [5]:
emb_layer = word2vec_emb
#else:
    #the learnt embedding layer will have the same vocab size as word2vec for comparaison reasons.
#emb_layer = nn.Embedding(num_embeddings=word2vec_emb.weight.shape[0], embedding_dim=300)

model = desperate_transformer(2048, emb_layer, 300, 1)
#model.init_weights()

#print(config["see_once"])
#count_parameters(model)

model.to(device)

desperate_transformer(
  (embedding): Embedding(8426, 300)
  (f_extr1): AttentionBlock(
    (key_gen): Sequential(
      (0): Linear(in_features=2048, out_features=600, bias=True)
      (1): ReLU()
      (2): Linear(in_features=600, out_features=600, bias=True)
      (3): ReLU()
      (4): Linear(in_features=600, out_features=300, bias=True)
      (5): ReLU()
    )
    (val_gen): Sequential(
      (0): Linear(in_features=2048, out_features=600, bias=True)
      (1): ReLU()
      (2): Linear(in_features=600, out_features=600, bias=True)
      (3): ReLU()
      (4): Linear(in_features=600, out_features=300, bias=True)
      (5): ReLU()
    )
    (query_gen): Sequential(
      (0): Linear(in_features=2048, out_features=600, bias=True)
      (1): ReLU()
      (2): Linear(in_features=600, out_features=600, bias=True)
      (3): ReLU()
      (4): Linear(in_features=600, out_features=300, bias=True)
      (5): Softmax(dim=-1)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): N

In [6]:
from torch.optim.lr_scheduler import StepLR
step_size = 1  # Number of epochs before adjusting the learning rate
gamma =  0.0001
def train(model):
    with wandb.init(project="dl2023_imagecaptioning", entity = "dl2023team") as run:
        
        run.name = f"LSTM&miniTransformer_v4"
        
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

        scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

        ############
        #Train loop:
        ############

        best_loss = float('inf')
        for epoch in range(num_epochs):
            model.train()
            training_losses = [] # renamed from epoch_losses
            progress_bar = tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch + 1}/{num_epochs}")
            
            for batch,(X1,X2,caption) in progress_bar:
            #for batch,(X1,X2) in enumerate(train_dataloader):
                optimizer.zero_grad()
                
                X1 = X1.to(device) 
                X2 = X2.to(device)
                out = model(X1,X2)
                #ic(out.shape)
                ref = X2
                #change ref to join batch and sequence dim
                ref = ref.reshape(ref.shape[0]*ref.shape[1])

                #same for the out logits
                #change ref to join batch and sequence dim
                out = out.reshape(out.shape[0]*out.shape[1],out.shape[2])
                
                loss = cross_entrop(out,ref)
                loss.backward()
                #torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
                optimizer.step()    
                training_losses.append(loss.item())
                progress_bar.set_postfix({'Batch Loss': loss.item()})
            
            scheduler.step()

            average_training_loss = sum(training_losses) / len(training_losses) # renamed from avg_loss
            wandb.log({'Train_Epoch_Loss': average_training_loss})

            if average_training_loss < best_loss:
                best_loss = average_training_loss
                torch.save(model.state_dict(), "LSTM&miniTransformer_v1.pt")
                wandb.save(f'LSTM&miniTransformer_v1.pt')
                #print(f"Model saved at {'{wandb.run.id}_LSTM&resnet18.pt'}")
        
        wandb.finish()

In [7]:
ic.disable()

In [8]:
train(model)


wandb: Currently logged in as: haradai (dl2023team). Use `wandb login --relogin` to force relogin


Epoch 1/200: 397it [08:33,  1.29s/it, Batch Loss=8.97]


KeyboardInterrupt: 

In [ ]:
iterable =iter(val_dataloader)

In [ ]:

X1,X2,caption = next(iterable)
X1 = X1.to(device)
X2 = X2.to(device)

In [ ]:
def generate_caption_tok(model,features,X2):
    features = features.to(device)
    tok = torch.tensor([[1]]).to(device)
    caption = []
    for i in range(X2.shape[1]):
        out = model(X1,tok)
        out = nn.functional.softmax(out)
        gen_word = torch.multinomial(out[0,0], 1)
        caption.append(gen_word[0])
        tok = gen_word.unsqueeze(0)     
    
    return torch.tensor(caption)

In [ ]:
def generate_caption(model,features):
    features = features.to(device)
    tok = torch.tensor([[1]]).to(device)
    caption = "bos"
    for i in range(40):
        out = model(X1,tok)
        out = nn.functional.softmax(out,dim=-1)
        gen_word = torch.multinomial(out[0,0], 1)
        caption += " "+vocabulary[gen_word]
        tok = gen_word.unsqueeze(0)
    
    print(caption)

In [ ]:
print(caption)
print(generate_caption(model,X1))

('bos man and woman kissing eos',)
bos sidecar prepairing swooping celebration brown mountaintops anything mastif tw boogie filled overcoat doorstep sushi encripted mercury nerf reclining physiques variety assisting scubba handkerchiefs cheer engraved old cylinder crooked threshold neckless flowered safe flag intently miasto sleeved windsurfing split tent headless
None
